In [13]:
%run ./work/lib.py

In [14]:
from pyspark import SparkConf, SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.types import StructType, StructField, LongType, StringType, DoubleType, DecimalType, TimestampType, DataType, IntegerType, FloatType
from pyspark.sql.functions import col, lit, trim, substring, concat, udf, upper, initcap
from datetime import datetime
import os

In [15]:
conf = SparkConf() \
    .setAppName("projeto_pbi") \
    .setSparkHome('./spark/home')

LAKE_HOME = os.getenv("LAKE_HOME", "/spark/home")

sc = SparkContext.getOrCreate(conf=conf)
spark = SparkSession(sc)

In [16]:
TABLE_NAME = 'dim_geolocation'

In [17]:
df = spark.read.csv('{}/dataset/e-commerce/01_extract/geolocation_dataset.csv'.format(LAKE_HOME), header=True)

df_geolocation = \
    df.select(
        col('geolocation_zip_code_prefix').cast(StringType()).alias('ID_ZIP_CODE'),
        col('geolocation_lat').cast(StringType()).alias('GEO_LATITUDE'),
        col('geolocation_lng').cast(StringType()).alias('GEO_LONGITUDE'),
        col('geolocation_city').cast(StringType()).alias('DES_CITY'),
        col('geolocation_state').cast(StringType()).alias('COD_STATE')
    )

sk = create_sk(spark=spark, df=df_geolocation, key_column_name='ID_ZIP_CODE', table_name=TABLE_NAME)

In [18]:
dim_geolocation = \
    df_geolocation \
        .withColumn('SK_GEOLOCATION', locate_sk(sk)(col('ID_ZIP_CODE')))

In [19]:
dim_geolocation = \
    dim_geolocation.select(
        col('SK_GEOLOCATION'),
        col('ID_ZIP_CODE'),
        col('GEO_LATITUDE'),
        col('GEO_LONGITUDE'),
        col('DES_CITY'),
        col('COD_STATE')  
    )

dim_geolocation.write.mode('overwrite').csv('{}/dataset/e-commerce/03_dim/{}.csv'.format(LAKE_HOME, TABLE_NAME), header=True)

In [20]:
spark.stop()